<h1> Power Digit Sum </h1> 

$2^{15} = 32768$ and the sum of the digits is $3+2+7+6+8 = 26$.

What is the sum of the digits of the number $2^{1000}$?

---


In [2]:
/// This is nCk assuming nC(k-1) is known
/// Since nC0 and nCn is always 1, this can always be used
fn binomial_coeffs(n: f64, k:f64, previous: f64) -> usize {
//     ((((n+1-k) as f64)/(k as f64))*(previous as f64)) as usize
//     let frac = ((n as f64) + 1.0 - (k as f64))/(k as f64);
//     println!("{}", frac);
//     (frac * previous as f64) as usize
    let frac = (n+1.0 -k)/k;
    (frac * previous).round() as usize
}

<h3> Attempt 1 </h3>

From Binomial theorem, I can express, 
\begin{equation}
    (x + y)^n = \sum_{k=0}^n {n \choose k} x^{n-k} y^k = \sum_{k=0}^n {n \choose k} x^{k} y^{n-k}
    \label{eq:1}
\end{equation}
What I want is the sum of digits, therefore, it would be desireable if I express a number in terms of 
\begin{equation*}
    \sum_{i=0}^n a_i x^i
\end{equation*}
where $a_i$ are coefficients and $x=10$.

From Equation (\ref{eq:1}), I can express it as, 
\begin{align*}
    (x - 8)^n =& \sum_{k=0}^n {n \choose k} x^{k} (-8)^{n-k} \\
    (10-8)^n = 2^n =& \sum_{k=0}^n {n \choose k}  (-8)^{n-k} 10^{k} \\
    =& \sum_{k=0}^n {n \choose k}  (-1)^{n-k} 2^{3(n-k)} 10^{k}
\end{align*}

If I had done this expansion and thought a little harder, I should've realised that this will result in numbers growing faster than if I calculated $2^n$ directly. 

In [3]:
:dep num = "*"
extern crate num;
// use num;

fn poly_coeffs(power:u32) -> Vec<i32> {
    let powusize: usize = power as usize;
    let maxlen: usize = powusize + 1;
    let mut out: Vec<i32> = vec![0 as i32; maxlen];
    let mut prev_binom_coeff: usize = 1;
    
    for i in 0..maxlen {
        out[i] = num::pow(-8i32, powusize-i)*(prev_binom_coeff as i32);
//         println!("{}, {}, {}", prev_binom_coeff, out[i], i);
        prev_binom_coeff = binomial_coeffs(powusize as f64, (i+1) as f64, prev_binom_coeff as f64); 
//         println!("{}, {}", prev_binom_coeff, i+1);
    }
    
    out
}

In [4]:
let sol = poly_coeffs(6);
println!("{:?}", sol);

[262144, -196608, 61440, -10240, 960, -48, 1]


<h3> Attempt 2 </h3>

Find 
\begin{equation}
    \label{eq:2}
    2^n = \sum_{k=0}^n {n \choose k}
\end{equation}
directly. 

This doesn't work as ${n \choose k}$ gets very large

In [5]:
fn binom_coeffs_half(power: u32) -> Vec<usize> {
    
    let mut midpoint: usize = (power/2 + (power)% 2) as usize;
    let mut is_symmetric: bool = true;
    
    if power%2 == 0 {
        midpoint += 1;
        is_symmetric = false;
    }
    
    println!("{} {} {}", power/2, (power)% 2, midpoint);
    
    let mut out: Vec<usize> = vec![0 as usize; midpoint]; 
    let mut prev_binom_coeff: usize = 1;
    out[0] = 2;
    
    for i in 1..midpoint {
        if is_symmetric || (!is_symmetric && i != midpoint -1) {
            out[i] = binomial_coeffs(power as f64, i as f64, prev_binom_coeff as f64); 
            prev_binom_coeff = out[i];
            out[i] = 2*out[i];
        } else {
            out[i] = binomial_coeffs(power as f64, i as f64, prev_binom_coeff as f64); 
            prev_binom_coeff = out[i];
        }
    }
    
    out
}

In [6]:
let sol = binom_coeffs_half(15);
println!("{:?}", sol);
let temp: usize = sol.iter().sum();
println!("{}", temp);

7 1 8
[2, 30, 210, 910, 2730, 6006, 10010, 12870]
32768


In [7]:
let k = (15.0*2.0_f64.log(10.0)) as usize +2;

// This is dumb and inefficient
for i in 1..k {
    let digit: f64 = sol.iter().map(|x| ((*x as f64)%10.0_f64.powi(i as i32))/10.0_f64.powi(i as i32)).sum();
    println!("{}", ((digit - digit.floor())*10.0) as usize );
};

8
6
7


<h3> Attempt 3 </h3>

The Vandermonde identity is 
\begin{equation}
    {n+m \choose r} = \sum_{k=0}^r {m \choose k} {n \choose r-k}
\end{equation}
To make it in the same form as Equation (2),
\begin{equation}
    \sum_{r=0}^{n+m} {n+m \choose r} = \sum_{r=0}^{n+m} \sum_{k=0}^r {m \choose k} {n \choose r-k} = 2^{n+m}
\end{equation}
A generalisation of the Vandermonde identity is, 
\begin{equation}
    \sum_{r=0}^{n_1 + n_2 + \ldots + n_p} {n_1 + n_2 + \ldots + n_p \choose r} = \sum_{r=0}^{n_1 + n_2 + \ldots + n_p} \sum_{k_1 + k_2 + \ldots + k_p = r} {n_1 \choose k_1} {n_2 \choose k_2} \ldots {n_p \choose k_p} = 2^{n_1 + n_2 + \ldots + n_p}
\end{equation}
Therefore, by storing the inner summation in a single entry of a vector, the individual terms to make up the final summation are known. 

Since $2^n$ can now be expressed as a summation that doesn't result in overflow, the property of sum of digits ($sd$) can be used. This property is given by, 
\begin{equation}
    sd(N + M) = sd(N) + sd(M) - 9\times q
\end{equation}
where $q$ is the carry over from the addition of $N + M$. This can be applied recursively to find the overall sum of digits.


In [23]:
fn multiplicative_binom_coefficient(n: u32, k: u32) -> u32 {
    let mut out: f64 = 1.0;
    let nfloat = n as f64;
    
    if (k == n) || (k == 0) {return out as u32}
    
    for i in 1..k+1 { 
        let ifloat = i as f64;
        out *= (nfloat + 1.0 - ifloat )/ (ifloat);
    }
//     println!("{}", out);
    out.round() as u32
}

In [31]:
extern crate itertools;
use itertools::Itertools;

fn get_unique_combinations(sum: u32, max_n_value: u32, p: u32) -> Vec<Vec<u32>> {
    
    let mut out: Vec<Vec<u32>> = Vec::new();
    let it = (0..max_n_value+1).combinations_with_replacement(p as usize); 
    
    for item in it {
        let combination_sum: u32 = item.iter().sum();
//         println!("{:?}", item);
//         println!("{}", combination_sum);
        
        if combination_sum == sum {
            out.push(item.clone());
            for perm in item.clone().into_iter().permutations(p as usize).unique() {
                if &perm != &item {out.push(perm.clone())}
            }
            
        }
    }
//     if i > sum {return out}
    out
}

In [11]:
fn set_k_value(r_val: u32, curr_k_sum: u32, curr_n: u32) -> u32 {
    if r_val > curr_k_sum + curr_n {
        return curr_n
    } else {
        return r_val - curr_k_sum
    }
}

In [59]:

fn inner_summation(n_val: u32, p: u32, r: u32) -> u32 {
    let mut sum_out = 0;
    let mut digit_sum = 0;
    
    let k_values = get_unique_combinations(r, n_val, p);
    
    for k_set in k_values.iter() {
//         println!("{:?}", k_set);
        let mut product = 1;
        for i in 0..p {
            let this_k: u32 = k_set[i as usize];
            if this_k != 0 || this_k != n_val {
                product *= multiplicative_binom_coefficient(n_val, this_k);
            }
//             println!("{} {} {} {}", r, n_val,  this_k, product);
        }
//         println!("{} {} {}", sum_of_digits(sum_out, 0), sum_of_digits(product, 0), 9*addition_carry_over(sum_out, product));
//         println!("{:?} {}", digit_sum, product);
        digit_sum += sum_of_digits(sum_out, 0) + sum_of_digits(product, 0) - 9*addition_carry_over(sum_out, product);
        sum_out += product;
    }
//     sum_out
//     println!("{:?}", sum_out);
    digit_sum
}

In [13]:
fn outer_summation(exponent: u32, max_n: u32) -> Vec<u32> {
    let quotient = exponent/max_n;
    if exponent % max_n != 0 {panic!("Exponent must be a multiple of n")}
    
    let mut inner_values: Vec<u32> = vec![0 as u32; (exponent+1) as usize];
    inner_values[0] = 1;
    inner_values[(exponent+1) as usize] = 1;
    
    for i in 1..exponent {
        inner_values[i as usize] = inner_summation(max_n, quotient, i);
    }
    
    inner_values
}

In [14]:
fn addition_carry_over(mut x: u32, mut y: u32) -> u32{
    let mut carry: u32 = 0; 
    while x != 0 || y != 0 {
        let digit_x = x % 10;
        let digit_y = y % 10;
        if digit_x + digit_y > 9 {carry += 1};
        x /= 10;
        y /= 10;
    }
    
    return carry 
}

In [15]:
fn sum_of_digits(mut input_num: u32, mut prev_sum: u32) -> u32 {
    if input_num/10 == 0{
        prev_sum += input_num % 10;
        return prev_sum
    } else {
        prev_sum += input_num % 10;
        return sum_of_digits(input_num/10, prev_sum)
    }
}

In [60]:
// let sol = sum_of_digits(1024, 0);
let sol = outer_summation(15, 5);
let solsum: u32= sol.iter().sum();
println!("{:?}, {}", sol, solsum);

[1, 12, 30, 62, 147, 201, 208, 288, 324, 208, 201, 147, 62, 30, 12, 1], 1934


In [17]:
println!("{}", addition_carry_over(256,69));

2
